# Initial Data Sourcing & Cleanup

In [1]:
# Import dependencies
import numpy as np
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup 
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo


In [12]:
# Use pandas to read the 21stCenturyBooks_Amazon csv
book_data = pd.read_csv("lerner_catalog_data.csv")
book_data

,ASIN,Product Title,Parent ASIN,ISBN-13,EAN,UPC,Release Date,List Price,Binding,Author / Artist,...,Product Group,Replenishment Code,Model / Style Number,Color,Color Count,Prep Instructions Required,Brand Code,Brand,Manufacturer Code,Parent Manufacturer Code
0,1541540220,"You Do You: Figuring Out Your Body, Dating, an...",1541540220,9.781540e+12,9.780000e+12,UNKNOWN,43774,37.32,library,"Mirk, Sarah",...,Books,NP,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
1,1541528131,Body 2.0: The Engineering Revolution in Medicine,1541528131,9.781540e+12,9.780000e+12,UNKNOWN,43774,37.32,library,"Latta, Sara",...,Books,NP,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
2,1541555546,Gun Violence: Fighting for Our Lives and Our R...,1541555546,9.781540e+12,9.780000e+12,UNKNOWN,43739,37.32,library,"Doeden, Matt",...,Books,NP,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
3,1541532384,Giraffe Extinction: Using Science and Technolo...,1541532384,9.781540e+12,9.780000e+12,UNKNOWN,43739,37.32,library,"Anderson, Tanya",...,Books,NP,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
4,1541519787,"The Dozier School for Boys: Forensics, Survivo...",1541519787,9.781540e+12,9.780000e+12,UNKNOWN,43711,37.32,library,"Murray, Elizabeth A.",...,Books,NP,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
5,1541540212,Mindfulness and Meditation: Handling Life With...,1541540212,9.781540e+12,9.780000e+12,UNKNOWN,43683,37.32,library,"Stewart, Whitney",...,Books,NP,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
6,1541528158,Vote!: Women's Fight for Access to the Ballot Box,1541528158,9.781540e+12,9.780000e+12,UNKNOWN,43683,37.32,library,"Frazer, Coral Celeste",...,Books,NP,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
7,1541540204,No More Excuses: Dismantling Rape Culture,1541540204,9.781540e+12,9.780000e+12,UNKNOWN,43466,37.32,library,"Keyser, Amber J.",...,Books,OB,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
8,1541539257,"Exposing Hate: Prejudice, Hatred, and Violence...",1541539257,9.781540e+12,9.780000e+12,UNKNOWN,43466,37.32,library,"Miller, Michael",...,Books,OB,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE
9,154152814X,Fake News: Separating Truth from Fiction,154152814X,9.781540e+12,9.780000e+12,UNKNOWN,43466,37.32,library,"Miller, Michael",...,Books,OB,UNKNOWN,UNKNOWN,0,—,TWENR,Twenty-First Century Books,LERNE,LERNE


In [13]:
# View column names
book_data.columns

Index(['ASIN', 'Product Title', 'Parent ASIN', 'ISBN-13', 'EAN', 'UPC',
       'Release Date', 'List Price', 'Binding', 'Author / Artist',
       'SITB enabled?', 'Apparel Size', 'Apparel Size Width', 'Product Group',
       'Replenishment Code', 'Model / Style Number', 'Color', 'Color Count',
       'Prep Instructions Required', 'Brand Code', 'Brand',
       'Manufacturer Code', 'Parent Manufacturer Code'],
      dtype='object')

In [14]:
# Create a new dataframe of only ASIN numbers
book_data_clean = book_data

# Remove unwanted columns
book_data_clean.drop(labels=['Product Title', 'Parent ASIN', 'ISBN-13', 'EAN', 'UPC',
       'Release Date', 'List Price', 'Binding', 'Author / Artist',
       'SITB enabled?', 'Apparel Size', 'Apparel Size Width', 'Product Group',
       'Replenishment Code', 'Model / Style Number', 'Color', 'Color Count',
       'Prep Instructions Required', 'Brand Code', 'Brand',
       'Manufacturer Code', 'Parent Manufacturer Code'], axis=1, inplace=True)

book_data_clean.head(20)

,ASIN
0,1541540220
1,1541528131
2,1541555546
3,1541532384
4,1541519787
5,1541540212
6,1541528158
7,1541540204
8,1541539257
9,154152814X


In [15]:
# Save century books cleaned dataframe to csv
book_data_clean.to_csv("ASIN_numbers.csv")

## Build query urls from ASIN numbers

In [16]:
#Create a dataframe to hold all of the API call data
col_name = ["URL"]
urls_df = pd.DataFrame(columns=col_name)

# Loop through the list of cities and add the data to the dataframe.
url = "https://search.barnesandnoble.com/bookSearch/isbnInquiry.asp?isbn="
row_count = 1

for index, row in book_data_clean.iterrows():
    query_asin = row["ASIN"]
    query_url = url + query_asin + "#/"
    url_data = requests.get(query_url)
    urls_df.set_value(index, "URL", query_url)
    row_count +=1

urls_df.head(20)

/anaconda3/envs/databootcampenv/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


,URL
0,https://search.barnesandnoble.com/bookSearch/i...
1,https://search.barnesandnoble.com/bookSearch/i...
2,https://search.barnesandnoble.com/bookSearch/i...
3,https://search.barnesandnoble.com/bookSearch/i...
4,https://search.barnesandnoble.com/bookSearch/i...
5,https://search.barnesandnoble.com/bookSearch/i...
6,https://search.barnesandnoble.com/bookSearch/i...
7,https://search.barnesandnoble.com/bookSearch/i...
8,https://search.barnesandnoble.com/bookSearch/i...
9,https://search.barnesandnoble.com/bookSearch/i...


In [17]:
# Save query urls to csv
urls_df.to_csv("url_list.csv")

# Scraping Book Data & Saving to MongoDB

In [18]:
def init_browser():

    executable_path = {'executable_path': '/anaconda3/pkgs/selenium-chromedriver-2.27-0/bin/chromedriver'}
    return Browser('chrome', **executable_path, headless=False)


In [26]:
browser = init_browser()
    
for url in urls_df["URL"]:
    
    browser.visit(url)
    time.sleep(1)
        
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
        
#     title = soup.find_all('h1', class_='pdp-header-title')
#     finaltitle = title[0].text
    
#     current_price = soup.find_all('span', class_='price current-price ml-0')
#     final_cprice = current_price[0].text
    
#     list_price = soup.find_all('input', id='listPrice')[0]['value']  
#     print(list_price)

browser.quit()

KeyboardInterrupt: 